In [9]:
import tensorflow as tf

In [10]:
!pip install -q transformers
!pip install -q sentencepiece
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import numpy as np
import pandas as pd


In [11]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
df_test = pd.read_csv('drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2test_news.csv')
df_test.text=df_test.text.astype(str)
df_test.head()

,text,label
0,مسجد تم إغلاقه وتشميعه من قبل الحكومة البريطان...,0
1,ساد التوتر في مخيم البداوي للاجئين الفلسطينيين...,1
2,مصدر الصورة Getty Images Image caption ميغان م...,1
3,خليجي 18 أنصف صاحب الأرض والجمهور ( الجزيرة نت...,1
4,لاعب من كينيا يعتزل كرة القدم نهائيا بعد تعرضه...,0


In [14]:
x_test = df_test.text
y_test = df_test.label

In [15]:
from transformers import pipeline
Model='vicgalle/xlm-roberta-large-xnli-anli'

classifier = pipeline("zero-shot-classification", model=Model,tokenizer=Model)

# First

## Sample Of Data

In [16]:
subset_news=[]
for news in df_test.text[:5]:
  pred_zero=classifier(news, ["خبر حقيقي", "خبر كاذب"])
  print(pred_zero)
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    subset_news.append(pred_zero["labels"][0])
    print(pred_zero["labels"][0])
  else:
    subset_news.append(pred_zero["labels"][1])
    print(pred_zero["labels"][1])


{'sequence': 'مسجد تم إغلاقه وتشميعه من قبل الحكومة البريطانية ، إلا أن صوت الآذان يزال يسمع عند كل صلاة معجزة من الله ، وقد قامت الحكومة بالتفتيش ولم يجدوا أثر لإنسان أو تسجيل وهذا هو الجامع . . إسمعوا الآذان', 'labels': ['خبر حقيقي', 'خبر كاذب'], 'scores': [0.9649884104728699, 0.03501162678003311]}
خبر حقيقي
{'sequence': 'ساد التوتر في مخيم البداوي للاجئين الفلسطينيين ، في شمال لبنان بضواحي مدينة طرابلس ، منذ مساء الأحد ، بعد تبادل لإطلاق النار بين الفصائل الفلسطينية ، ما أدى لجرح اثنين . واستمر الاشتباك بين عناصر من القيادة العامة وعناصر من فتح الانتفاضة ، طوال الليل على فترات متقطعة ، حيث كان يسمع أزيز الرصاص بين حين وآخر ، ما أدى إلى إصابة شخصين بجراح . إقرأ المزيد وتعمل القيادات الفلسطينية على تهدئة الوضع والاتصالات جارية مع جميع الأطراف في محاولة لوقف الاشتباك ، حتى تمت السيطرة على الوضع فجر الاثنين . وأعلنت مدارس مخيم البداوي اليوم عطلة اضطرارية حفاظا على أمن الطلاب وسلامتهم وخوفا من تدهور الوضع مجددا . وتزامنا مع اندلاع الاشتباكات ، سير الجيش اللبناني دوريات مشددة عند أطراف مخ

In [17]:
subset_news[:5]

['خبر حقيقي', 'خبر حقيقي', 'خبر حقيقي', 'خبر حقيقي', 'خبر كاذب']

## All Data

In [ ]:
i=0
y_pred_new=[]
for news in df_test.text:
  i=i+1
  pred_zero=classifier(news, ["خبر حقيقي", "خبر كاذب"])
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    y_pred_new.append(pred_zero["labels"][0])
  else:
    y_pred_new.append(pred_zero["labels"][1])

  print(str(i) +"-"+news)

y_pred_new[:5]

1-مسجد تم إغلاقه وتشميعه من قبل الحكومة البريطانية ، إلا أن صوت الآذان يزال يسمع عند كل صلاة معجزة من الله ، وقد قامت الحكومة بالتفتيش ولم يجدوا أثر لإنسان أو تسجيل وهذا هو الجامع . . إسمعوا الآذان
2-ساد التوتر في مخيم البداوي للاجئين الفلسطينيين ، في شمال لبنان بضواحي مدينة طرابلس ، منذ مساء الأحد ، بعد تبادل لإطلاق النار بين الفصائل الفلسطينية ، ما أدى لجرح اثنين . واستمر الاشتباك بين عناصر من القيادة العامة وعناصر من فتح الانتفاضة ، طوال الليل على فترات متقطعة ، حيث كان يسمع أزيز الرصاص بين حين وآخر ، ما أدى إلى إصابة شخصين بجراح . إقرأ المزيد وتعمل القيادات الفلسطينية على تهدئة الوضع والاتصالات جارية مع جميع الأطراف في محاولة لوقف الاشتباك ، حتى تمت السيطرة على الوضع فجر الاثنين . وأعلنت مدارس مخيم البداوي اليوم عطلة اضطرارية حفاظا على أمن الطلاب وسلامتهم وخوفا من تدهور الوضع مجددا . وتزامنا مع اندلاع الاشتباكات ، سير الجيش اللبناني دوريات مشددة عند أطراف مخيم البداوي ومداخله ، ورفع جهوزيته وأقام الحواجز لمنع توسع الإشكال إلى خارج المخيم . وقالت الوكالة الوطنية الرسمية ، إن تلاسنا 

## Accuercy

In [ ]:
convert = lambda x : 0 if x=='خبر كاذب' else 1

In [ ]:
y_pred=[]
for news in y_pred_new:
  y_pred.append(convert(news))

In [ ]:
y_pred[:5]

In [ ]:
y_test[:5]

In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))
print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

## Example

In [ ]:
#real
pred_zero=classifier('«خارج عن السيطرة».. صاروخ أطلقته الصين يوم الخميس قد يسقط على الأرض خلال أيام',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)


In [ ]:
#fake
pred_zero=classifier('✋✋ مين ضايل لم يزاود على السلطة و الاجهزة الامنية 👇👇👇 قناة 13 العبرية: زوجة منفذ عمـ ـلية زعتره منتصر شلبي أبرمت صفقة مع السفارة الأمريكية في القدس تضمنت تسليم زوجها مقابل عدم قتله، وهي من أعطت المعلومة عن مكان وجوده.وتسلم الايادي لوحوش البحرية وتحيا مصر',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

In [ ]:
#fake
pred_zero=classifier('عااااااجل القوات البحرية وخفر السواحل تشتبه بإحدى مراكب الصيد بالمياه الإقليمية بالبحر الابيض وبحق التفتي والزيارة وجدت وسط العمال ضابط مخابرات تركي وسبع جنود وبتقنين الاجرائات تم اصطحابهم الي النيابة العسكرية مين هيقول لقردو مبروك وتسلم الايادي لوحوش البحرية وتحيا مصر',["خبر حقيقي", "خبر كاذب"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

# Second

## sample of Data

In [ ]:
subset_news=[]
for news in df_test.text[:5]:
    pred_zero=classifier(news, ["real news ", "fake news"])
    print(pred_zero)
    if(pred_zero["scores"][0]> pred_zero["scores"][1]):
      subset_news.append(pred_zero["labels"][0])
      print(pred_zero["labels"][0])

    else:
      subset_news.append(pred_zero["labels"][1])
      print(pred_zero["labels"][1])

subset_news[:5]

## All Data

In [ ]:
i=0
y_pred_new=[]
for news in df_test.text:
  i=i+1
  pred_zero=classifier(news, ["real news ", "fake news"])
  if(pred_zero["scores"][0]> pred_zero["scores"][1]):
    y_pred_new.append(pred_zero["labels"][0])
  else:
    y_pred_new.append(pred_zero["labels"][1])
    
  print(str(i) +"-"+news)


y_pred_new[:5]

## Accuercy

In [ ]:
convert = lambda x : 0 if x=='fake news' else 1

In [ ]:
y_pred=[]
for news in y_pred_new:
  y_pred.append(convert(news))
   

In [ ]:
y_pred[:5]

In [ ]:
print("Accuracy", accuracy_score(y_test, y_pred))
print("\nConfusion matrix\n",confusion_matrix(y_test,y_pred))
print("\nClassification report\n",classification_report(y_test,y_pred))

## Sample

In [ ]:
pred_zero=classifier('«خارج عن السيطرة».. صاروخ أطلقته الصين يوم الخميس قد يسقط على الأرض خلال أيام', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

In [ ]:
pred_zero=classifier('✋✋ مين ضايل لم يزاود على السلطة و الاجهزة الامنية 👇👇👇 قناة 13 العبرية: زوجة منفذ عمـ ـلية زعتره منتصر شلبي أبرمت صفقة مع السفارة الأمريكية في القدس تضمنت تسليم زوجها مقابل عدم قتله، وهي من أعطت المعلومة عن مكان وجوده.وتسلم الايادي لوحوش البحرية وتحيا مصر', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)

In [ ]:
pred_zero=classifier('عااااااجل القوات البحرية وخفر السواحل تشتبه بإحدى مراكب الصيد بالمياه الإقليمية بالبحر الابيض وبحق التفتي والزيارة وجدت وسط العمال ضابط مخابرات تركي وسبع جنود وبتقنين الاجرائات تم اصطحابهم الي النيابة العسكرية مين هيقول لقردو مبروك وتسلم الايادي لوحوش البحرية وتحيا مصر', ["real news ", "fake news"])

SCORES = pred_zero["scores"]
labels=pred_zero["labels"]
print(SCORES)
print(labels)